In [102]:
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
import math
import numba
from numba import jit, njit
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("dataset.csv")

In [4]:
x,y = list(df['Subject 1']), list(df['Subject 2'])

In [18]:
@jit(parallel = True, fastmath=True)
def mean(arr):
    return sum(arr)/len(arr);

In [103]:
mean_x, mean_y = mean(x), mean(y)

In [80]:
@jit(parallel=True,fastmath=True)
def get_bias_and_constant(arr, l, mean_x=mean_x, mean_y=mean_y):
    sn, sd=0,0;
    for i in range(len(arr)):
        sn+=(arr[i]-mean_x)*(l[i]-mean_y)
        sd+=(arr[i]-mean_x)**2
    bias, constant = 0,0
    bias = sn/sd
    constant = mean_y - mean_x*bias
    print("intercept : {}, m : {}".format(constant, bias))
    
    return bias, constant

In [104]:
m,c = get_bias_and_constant(arr=x, l=y, mean_x=mean_x, mean_y=mean_y)

intercept : -1.6460674157303181, m : 1.0749063670411978


In [128]:
@jit(parallel=True, fastmath=True)
def metadata_stats(x=x,y=y,mean_x = mean_x, mean_y = mean_y,pred_y=pred_y,m=m,c=c):
    res_sum_of_sq, tot_sum_of_sq, sq_err = 0,0,0
    for i in range(len(x)):
        res_sum_of_sq+=(y[i]-pred_y[i])**2
        tot_sum_of_sq+=(y[i]-mean_y)**2
    r_sq = math.sqrt(res_sum_of_sq/(len(x)-2))
    sq_err = 1 - (res_sum_of_sq/tot_sum_of_sq)
    print("Intercept : {}\nBias : {}".format(c,m))
    print("Squared Error : {}".format(sq_err[0]))

In [57]:
df['Predicted'] = df['Subject 1'].apply(lambda x : (m*x +c))

In [61]:
lr = LinearRegression()
lr.fit(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [64]:
pred_y = lr.predict(np.array(x).reshape(-1,1))

In [72]:
pr_y = [pred_y[i][0] for i in range(len(pred_y))]

In [82]:
df['Library Prediction']=pr_y

In [98]:
@jit(fastmath=True)
def calculate_error(col1, col2):
    return col1-col2

In [99]:
df['Absolute error']=df.apply(lambda x : calculate_error(x['Predicted'], x['Library Prediction']), axis=1)

In [100]:
df.head(5)

,Unnamed: 0,Subject 1,Subject 2,Predicted,Library Prediction,Absolute error
0,0,24,26,24.151685,24.151685,3.552714e-15
1,1,26,29,26.301498,26.301498,0.000000e+00
2,2,27,26,27.376404,27.376404,0.000000e+00
3,3,21,23,20.926966,20.926966,0.000000e+00
4,4,25,26,25.226592,25.226592,0.000000e+00


In [130]:
print("Metadata Stats (Predicted):\n")
metadata_stats()

Metadata Stats (Predicted):

Intercept : -1.6460674157303181
Bias : 1.0749063670411978
Squared Error : 0.6140284505042108


In [132]:
print("Metadata Stats (Library) :\n")
print("Intercept  : {}\nBias : {}\nSquared Error : {}".format(lr.intercept_[0],lr.coef_[0][0],lr.score(np.array(x).reshape(-1,1),np.array(y).reshape(-1,1))))

Metadata Stats (Library) :

Intercept  : -1.6460674157303359
Bias : 1.0749063670411985
Squared Error : 0.6140284505042108
